<a href="https://colab.research.google.com/github/x-legion/legions-sd-notebooks/blob/main/legions_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Install ```Dependencies```

import os
from IPython.display import clear_output

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf
!pip install pyrfc6266

repo_dir = "/content/ComfyUI"

!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd {repo_dir}
!pip install -q -r requirements.txt

%cd {repo_dir}/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Stability-AI/stability-ComfyUI-nodes.git
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux/
# !git clone https://github.com/WASasquatch/was-node-suite-comfyui.git
!git clone https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes.git
!git clone https://github.com/EllangoK/ComfyUI-post-processing-nodes.git
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive
!git clone https://github.com/BlenderNeko/ComfyUI_TiledKSampler.git
!git clone https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb.git
!git clone https://github.com/BlenderNeko/ComfyUI_Noise.git
!git clone https://github.com/failfa-st/failfast-comfyui-extensions.git
!git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui.git
# !git clone https://github.com/RockOfFire/ComfyUI_Comfyroll_CustomNodes.git
# !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git
!git clone https://github.com/SeargeDP/SeargeSDXL.git
!git clone https://github.com/BlenderNeko/ComfyUI_Cutoff
!git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui
!git clone https://github.com/Davemane42/ComfyUI_Dave_CustomNode
# !git clone https://github.com/pythongosssss/ComfyUI-WD14-Tagger.git
!git clone https://github.com/ssitu/ComfyUI_restart_sampling
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts
# !git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors.git
!git clone https://github.com/twri/sdxl_prompt_styler.git
!git clone https://github.com/Ser-Hilary/SDXL_sizing.git
!git clone https://github.com/Extraltodeus/noise_latent_perlinpinpin.git
!git clone https://github.com/city96/SD-Latent-Upscaler.git
!git clone https://github.com/WASasquatch/PPF_Noise_ComfyUI.git
!git clone https://github.com/cubiq/ComfyUI_SimpleMath.git
!wget https://huggingface.co/faisalhr1997/comfyui/resolve/main/SDXLAspectRatio.py
!wget https://huggingface.co/faisalhr1997/comfyui/resolve/main/ksampler_sdxl.py

# !python /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors/install.py
# !pip install -r {repo_dir}/custom_nodes/ComfyUI-WD14-Tagger/requirements.txt
!pip install -r /content/ComfyUI/custom_nodes/comfyui_controlnet_aux/requirements.txt

clear_output()
print("\033[34m>> Finished installing Dependencies\033[0m")

In [ ]:
# @title 2. Download ```Models```

import os, re, csv, requests, pyrfc6266
from IPython.display import clear_output

%cd {repo_dir}

model_dir = "models/checkpoints" # @param ["models/checkpoints", "models/vae", "models/loras", "models/embeddings", "models/controlnet", "models/upscale_models", "models/clip_vision"] {allow-input: true}
download_links = "" # @param ["https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl_vae.safetensors, https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors, https://huggingface.co/Linaqruf/stolen/resolve/main/vae/ClearVAE_V2.2.safetensors", "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", "https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/lollypop.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/DPED.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_foolhardy_Remacri.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_Nickelback_70000G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-YandereNeoXL_200k.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-Superscale-SP_178000_G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-UltraSharp.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-AnimeSharp.pth", "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-canny-rank128.safetensors, https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-depth-rank128.safetensors, https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-recolor-rank128.safetensors, https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-sketch-rank128-metadata.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors, https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors", "https://huggingface.co/stabilityai/control-lora/resolve/main/revision/clip_vision_g.safetensors"] {allow-input: true}

links = list(csv.reader([download_links]))[0]
session = requests.session()
for link in links:
  response = session.get(link, stream=True)
  file_name = pyrfc6266.requests_response_to_filename(response)
  print(f"Downloading {file_name} to {model_dir}")
  !aria2c --optimize-concurrent-downloads --console-log-level=error --summary-interval=10 -j5 -x16 -s16 -k1M -c -d {model_dir} {link} -o {file_name}
  clear_output()

for dir in ["checkpoints", "vae", "loras", "embeddings", "controlnet", "upscale_models","clip_vision"]:
  print(f"\033[31m>> {dir}\033[0m")
  !ls models/{dir}


In [ ]:
# @title 3. Run ```ComfyUI``` [*](https://huggingface.co/faisalhr1997/comfyui/resolve/main/2workflow.json)



import os
import subprocess
import threading
from threading import Timer
import time
import socket
import urllib.request
import re
from IPython.display import clear_output
from google.colab import drive


launch_arguments = "--dont-print-server --enable-cors-header --preview-method latent2rgb --highvram --fp16-vae --disable-metadata" # @param [""] {allow-input: true}
drive_output = True # @param {type:"boolean"}
tunnel = "Cloudflared" # @param ["Cloudflared", "Localtunnel", "Iframe"]
# optional_ngrok_token="2GJxsLpqP2RMqpALqo94vrQMckA_yFECTsEBCNJtxkmykc2t" #@param{type:"string"}
# optional_ngrok_region = "eu" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]
# auth_token = optional_ngrok_token
# region = optional_ngrok_region

if drive_output:
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive
    !mkdir comfyui_output
    launch_arguments += " --output-directory /content/drive/MyDrive/comfyui_output"

%cd {repo_dir}

if tunnel == "Cloudflared":
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb


  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
      l = line.decode()
      if "trycloudflare.com " in l:
        clink = l[l.find("http"):]
        tlink = re.sub(r"[^\w:/.-]", "", clink) # [^\w:/.] matches any character that is not a word character, a colon, a slash, or a dot
        cblink = tlink.replace("https://", "")
        print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
        print("This is the URL to access ComfyBOX:")
        print(f"https://faisalhr1997-comfybox.hf.space/index.html?url={cblink}&port=443")


      #print(l, end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py {launch_arguments}

elif tunnel == "Localtunnel":
  !npm install -g localtunnel
  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py {launch_arguments}

# elif tunnel == "Ngrok":
#   try:
#     from pyngrok import conf,ngrok
#   except:
#     with capture.capture_output() as _:
#       !pip install -qqqq --upgrade setuptools
#       !pip install -qqqq -U pyngrok
#     from pyngrok import conf,ngrok

#   def ngrok_tunnel(port,queue,auth_token,region="jp"):
#     tunnels = ngrok.get_tunnels()
#     if tunnels : ngrok.kill()
#     conf.get_default().region = region
#     ngrok.set_auth_token(auth_token)
#     url = ngrok.connect(port)
#     queue.put(url)

#   ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, optional_ngrok_token, optional_ngrok_region))
#   ngrok_thread.start()
#   ngrok_thread.join()
#   !python main.py {launch_arguments}

elif tunnel == "Iframe":
  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    from google.colab import output
    output.serve_kernel_port_as_iframe(port, height=1024)
    print("to open it in a window you can open this link here:")
    output.serve_kernel_port_as_window(port)

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py {launch_arguments}

In [ ]:
# @title Show ```launch_arguments```

%cd {repo_dir}

!python main.py --help
